In [0]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
X = []
y = []
datadir = '../input/braille-character-dataset/Braille Dataset/Braille Dataset'
for image in os.listdir(datadir):
    x = cv2.imread('../input/braille-character-dataset/Braille Dataset/Braille Dataset/{}'.format(image))
    x = cv2.cvtColor(x,cv2.COLOR_BGR2GRAY)
    X.append(x.tolist())
    y.append(ord(image[0]))
X = np.asarray(X)
X = (X-127.5)/127.5
X = np.expand_dims(X, axis=3)
y = np.asarray(y)
y = y - ord('a')
y = to_categorical(y)

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,BatchNormalization,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
def braille():
    inputs = Input(shape=(28,28,1)) 
    X = Conv2D(64,3,padding='same',activation='relu')(inputs)
    X = BatchNormalization()(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(128,3,padding='same',activation='relu')(X)
    X = BatchNormalization()(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(256,3,padding='same',activation='relu')(X)
    X = BatchNormalization()(X)
    X = Flatten()(X)
    X = Dense(256,activation='relu')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Dense(64,activation='relu')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Dense(26,activation='softmax')(X)
    return Model(inputs,X)
model = braille()
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
model_ckpt = ModelCheckpoint('BrailleNet.h5',save_best_only=True)
reduce_lr = ReduceLROnPlateau(patience=8,verbose=0)
early_stop = EarlyStopping(patience=20,verbose=1)
model.fit(X,y,epochs=100,callbacks=[model_ckpt,reduce_lr,early_stop])

In [0]:
idx = np.random.randint(y.shape[0])
plt.imshow(X[idx].reshape(28,28),cmap='Greys_r')
plt.colorbar()
print(np.argmax(y[idx]))
print(np.argmax(model.predict(np.expand_dims(X[idx], axis=0))))

In [0]:
test = cv2.imread('../input/braille/goal.png')
test = cv2.cvtColor(test,cv2.COLOR_BGR2GRAY)
test = test.astype(float)
test = cv2.resize(test,(825,2052))
test = -(test-51)/51

In [0]:
plt.imshow(test)

In [0]:
X_test = cv2.resize(test[0:36,0:25],(28,28))
X_test = X_test.reshape(1,28,28,1)
plt.imshow(X_test.reshape(28,28),cmap='Greys_r')
plt.colorbar()
print(chr(np.argmax(model.predict(X_test))+ord('a')))
print(model.predict(X_test))

In [0]:
isnum = False

for i in range(57):
    for j in range(33):
        if j < 10:
            X_test = cv2.resize(test[36*i:36*(i+1),5+25*j:5+25*(j+1)],(28,28))
        else:
            X_test = cv2.resize(test[36*i:36*(i+1),25*j:25*(j+1)],(28,28))
        X_test = X_test.reshape(1,28,28,1)
        predict = chr(np.argmax(model.predict(X_test))+ord('a'))
        if predict == 'y':
            isnum = True
            iscap = False
            continue
        elif predict == 'u':
            iscap = True
            isnum = False
            continue
        else:
            if isnum == True:
                print(str((np.argmax(model.predict(X_test))+1)%10),end='')
                isnum = False
            elif iscap == True:
                print(str(chr(np.argmax(model.predict(X_test))+ord('A'))),end='')
                iscap = False
            else:
                print(str(chr(np.argmax(model.predict(X_test))+ord('a'))),end='')